In [2]:
#hiddencell
from pbl_tools import *

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(fname = 'NotoSansKR-Regular.otf', name = 'NotoSansKR')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='NotoSansKR')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 스테이지 5. 고급 데이터 전처리 및 차원 축소를 통한 모델 학습과 평가
## 도입
이번 스테이지에서는 고급 데이터 전처리와 차원 축소를 통한 모델 학습과 평가에 집중합니다.   
이전 스테이지에서는 데이터의 기본 구조와 피처 상호 작용을 다뤘으며,   
이번에는 더 깊은 수준의 데이터 전처리와 차원 축소 기술을 활용하여 모델을 개선하고자 합니다.

## 학습 목표
- 교육 수준(education)과 결혼 상태(marital.status)를 수준 카테고리로 재분류할 수 있다.
- 파생변수를 생성하여 데이터를 보다 의미 있게 표현할 수 있다.
- 범주형 변수를 원-핫 인코딩하여 모델 학습에 활용할 수 있다.
- PCA (주성분 분석)를 활용하여 데이터의 차원을 축소하고 주요 특성을 추출할 수 있다.
- PCA 주성분의 분산 설명량을 확인하고 누적 설명량을 고려하여 주성분 개수를 선택할 수 있다.
- 데이터를 표준화하고 PCA 모델을 적용하여 데이터를 변환할 수 있다.
- RandomForestClassifier 모델을 활용하여 데이터를 분석하고 성능을 평가할 수 있다.

# 1. pandas를 이용해 csv 파일 읽어오기
[문제 1]  
`Pandas` 라이브러리(library)를 가져와보세요.  
그리고 `train.csv`, `test.csv`, `sample_submission.csv` 파일을 각각 train, test, submission 변수로 읽어오세요.  
아래 빈칸을 채워주세요.  

In [2]:
import pandas as pd  

train = pd.___('train.csv')  
test = pd.___('test.csv')  
submission = pd.___('sample_submission.csv')

In [21]:
#checkcode
ensure_vals(globals(), 'train', 'test', 'submission')
@check_safety
def check(
    user_train: pd.DataFrame,
    user_test: pd.DataFrame,
    user_submission: pd.DataFrame
):
    c_point1 = hasattr(user_train, 'tail')
    c_point2 = hasattr(user_test, 'tail')
    c_point3 = hasattr(user_submission, 'tail')

    if c_point1 and c_point2 and c_point3:
        return True
    else:
        return False

check(train, test, submission)

True

# 2.교육(education) 수준 카테고리 재분류

데이터 전처리 과정에서는 종종 특성의 값을 재구분하는 작업이 필요합니다.   
이는 카테고리형 변수의 값들을 더 큰 범주로 묶어 데이터의 복잡도를 줄이고,   
모델 학습에 유용한 정보를 보다 명확하게 표현하는데 도움이 됩니다.

우리의 데이터셋에서 'education' 특성은 사람들의 교육 수준을 나타내는 여러 가지 값으로 구성되어 있습니다.   
그러나 이러한 각각의 값들이 모델 학습에 동일하게 중요하지 않을 수 있으며,   
비슷한 교육 수준을 가진 카테고리들은 실질적으로 같은 의미를 가질 가능성이 있습니다.

따라서 'education' 특성의 일부 값을 새로운 범주로 묶어보도록 하겠습니다.   
이렇게 함으로써 우리는 데이터 내에서 비슷한 의미를 가진 여러 개의 범주를 단순화하여 모델 성능을 향상시키고자 합니다.

[문제 2]

- `replace()` 함수를 사용하여 초등학교 교육 수준('Preschool', '1st-4th', '5th-6th')과 중학교 초기('7th-8th') 및 후기('9th'), 고등학교 초기('10th', '11th') 및 후기('12th') 등을 포함하는 여러 개의 범주를 하나인 `LowEducation`으로 재구분해주세요.

- 일부 고등 교육 수준(대학 일부 코스와 전문 대학 코스)은 `SomeHigherEd`로,    
석사 학위와 전문 학위 프로그램은 모두 `Masters`로 재구분하였습니다.

In [5]:
train['education'].___(['Preschool', '10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th'], 'LowEducation', inplace=True)
train['education'].replace(['Some-college', 'Assoc-acdm', 'Assoc-voc'], 'SomeHigherEd', inplace=True)
train['education'].replace(['Masters', 'Prof-school'], ['Masters', 'Masters'], inplace=True)

In [23]:
#checkcode
ensure_vals(globals(), 'train')
@check_safety
def check(
    df: pd.DataFrame,
    col: str,
    val1: str,
    val2: str,
    val3: str
):
    c_point0 = val1 in df[col].unique()
    c_point1 = val2 in df[col].unique()
    c_point2 = val3 not in df[col].unique()

    if c_point0 and c_point1 and c_point2:
        return True
    else:
        return False

check(train, 'education', 'LowEducation', 'SomeHigherEd', 'Prof-school')

True

# 3.결혼 상태 (marital.status) 수준 카테고리 재분류

Never.married는 결혼을 한 번도 하지 않은 상태를 나타내며, Married.spouse.absent는 배우자가 부재한 상태를 나타냅니다.   
이 두 상태를 통합해주겠습니다.

Married.AF.spouse는 미국 공군의 배우자와 결혼한 상태를 나타내며, Married.civ.spouse는 미국 시민과 결혼한 상태를 나타냅니다.
이 두 상태를 통합해주겠습니다.

Separated는 별거 중인 상태를 나타내며, Divorced는 이혼한 상태를 나타냅니다.  
이 두 상태를 통합해주겠습니다.

[문제 3]  

- `train` 데이터셋의 `marital.status` 열에서 `Never.married`와 `Married.spouse.absent`의 값을 `UnmarriedStatus`로 대체해주세요.  
`inplace=True` 매개변수를 사용하여 원본 데이터프레임을 변경하도록 설정 해주세요.
- marital.status 열에서 `Married.AF.spouse`와 `Married.civ.spouse`의 값을 Married로 대체해주세요.
- marital.status 열에서 `Separated와 Divorced`의 값을 `MarriageEnded`로 대체해주세요.

In [6]:
train['marital.status'].___(['Never.married', 'Married.spouse.absent'], 'UnmarriedStatus', inplace=___)
train['marital.status'].replace(['Married.AF.spouse', 'Married.civ.spouse'], '___', inplace=True)
___[___].___([___, ___], ___, inplace=___)

In [29]:
#checkcode
ensure_vals(globals(), 'train')
@check_safety
def check(
    df: pd.DataFrame,
    col: str,
    val1: str,
    val2: str,
):
    c_point0 = val1 in df[col].unique()
    c_point1 = val2 not in df[col].unique()

    if c_point0 and c_point1:
        return True
    else:
        return False

check(train, 'marital.status', 'MarriageEnded', 'Separated')

True

# 4.나이(age)와 주당 근무시간(hours.per.week)을 곱한 새로운 특성 생성


데이터 분석 과정에서는 새로운 특성을 생성함으로써 모델의 성능을 향상시키는 `특성 엔지니어링(feature engineering)`이 필요합니다.   
이는 기존 데이터에서 추가적인 정보를 추출하거나, 데이터 간의 복잡한 관계를 캡처하는 데 도움이 됩니다.

우리의 경우, `age`와 `hours.per.week`라는 두 가지 특성 사이에 중요한 상호작용(interaction)이 있음을 발견하였습니다.   
이전 단계인 stage3에서 scatterplot을 그려본 결과, 나이와 주당 근무시간 사이에 분명한 패턴 혹은 관계가 보여,   
이 두 변수를 조합하여 새로운 특성 `age-hours`를 생성하기로 결정하였습니다.

새로운 `age-hours` 특성은 개개인의 나이와 그들이 일하는 시간 사이의 복합적인 영향력을 반영할 것입니다.   
이렇게 함으로써 우리 모델은 나이와 근무 시간 각각만 고려하는 것보다 더 많은 정보를 얻어 성능 개선에 도움을 줄 수 있게 될 것입니다.

In [30]:
train['age-hours'] = train['age']*train['hours.per.week']
train.head(3)

,ID,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,age-hours
0,TRAIN_0000,75,Self-emp-not-inc,218521,SomeHigherEd,10,Married-spouse-absent,Craft-repair,Not-in-family,White,Male,0,0,30,United-States,0,2250
1,TRAIN_0001,23,Private,194102,Bachelors,13,Never-married,Exec-managerial,Unmarried,White,Male,0,0,40,United-States,0,920
2,TRAIN_0002,34,Private,238305,SomeHigherEd,10,Married-civ-spouse,Other-service,Wife,White,Female,0,1628,12,NaN,0,408


# 5.데이터 분할 및 train/valid 데이터 준비

[문제 4]
- 독립변수 x에는 train 데이터셋에서 target,education 을 제외한 나머지 모든 열의 값들로 설정해 주세요.
- 종속변수 y에는 train 데이터셋에서 target 열의 값으로 설정해 주세요.
- 독립변수 x, 종속변수 y 데이터를 훈련 세트(train)와 검증 세트(valid)로 (7:3의 비율로) 나누어 보세요.

In [314]:
x = ...
y = ...

from sklearn.model_selection import train_test_split
___, ___, ___, ___ = ___(x, y, ___ = 0.3, random_state = 42)

In [33]:
#checkcode
ensure_vals(globals(), 'x_valid', 'y_valid')
@check_safety
def check(
        user_answer_x : str,
        user_answer_y : str,
):
    c_point0 = hasattr(user_answer_x, 'head')
    c_point1 = hasattr(user_answer_y, 'head')

    if c_point0 and c_point1:
        return True
    else:
        return False

check(x_valid,y_valid)

True

# 6.결측치 처리와 불필요한 열 제거를 위한 데이터 전처리

[문제 5]

- 결측치 처리를 위한 `SimpleImputer` 클래스를 가져오세요.    
- `SimpleImputer` 객체를 생성하고, 결측치를 처리할 때 해당 특성에서 가장 자주 발생하는 값으로 결측치를 대체해주세요.
- 학습 데이터셋인 `x_train`의 'occupation'과 'workclass' 열에 대해 결측치를 `최빈값`으로 채워주세요.
- 검증 데이터셋인 `x_valid`의 'occupation'과 'workclass' 열에 대해 결측치를 `최빈값`으로 채워주세요.
- x_train과 x_valid 데이터셋에서 불필요한 피처인 `'native.country','ID'` 을 제거해 주세요.

In [316]:
from sklearn.impute import ___

# SimpleImputer를 사용하여 결측치를 최빈값으로 보간
imputer = ___(strategy='___')
x_train[['occupation','workclass']] = ___.___(...)
x_valid[['occupation','workclass']] = ___.___(...)

# 불필요한 열 제거
x_train = x_train.___(...)
x_valid = x_valid.___(...)

In [43]:
#checkcode
ensure_vals(globals(), 'x_train', 'imputer')
@check_safety
def check(
    df: pd.DataFrame,
    encoder: SimpleImputer,
    not_col: str,
    use_col1: str,
    use_col2: str
):
    c_point0 = not_col not in df.columns
    c_point1 = use_col1 in encoder.feature_names_in_
    c_point1 = use_col2 in encoder.feature_names_in_

    if c_point0 and c_point1:
        return True
    else:
        return False

check(x_train, imputer, 'native.country', 'workclass', 'occupation')

True

# 7.범주형 변수 원-핫 인코딩

이전 스테이지에서는 모든 범주형 변수를 레이블 인코딩(label encoding)을 통해 수치형으로 변환하였습니다.   
이 방법은 각 범주에 고유한 정수 값을 할당하여 `순서가 있는 특성(ordinal features)`을 처리하는 데 유용합니다.

그러나 'race', 'sex', 'marital.status'와 같은 특성들은 `순서가 없는(nominal)` 범주형 변수입니다.   
이런 경우, 각 범주를 독립적인 이진 특성으로 변환하는 `원-핫 인코딩(one-hot encoding)`이 더 적합합니다.   
원-핫 인코딩은 모델에게 잘못된 정보(예: 하나의 범주가 다른 것보다 큰 것으로 해석될 수 있음)를 제공하는 것을 방지할 수 있습니다.

sklearn의 `OneHotEncoder`를 사용하여 'race', 'sex', 'marital.status' 세 가지 특성에 대해 원-핫 인코딩을 적용하겠습니다.

[문제 6]
- scikit-learn의 preprocessing 모듈에서 `OneHotEncoder` 클래스를 불러오세요.
- `OneHotEncoder` 객체를 생성합니다.   
`sparse=False` 옵션은 희소 행렬(sparse matrix)이 아닌 밀집 행렬(dense matrix) 형태로 데이터를 반환하도록 설정합니다.
- 학습 데이터셋(`x_train`)의 `'race', 'sex', 'marital.status'` 열을 선택하고,   
`fit_transform` 메서드를 사용하여 원-핫 인코딩을 수행합니다.   
이 작업은 학습 데이터에 대해 모델을 훈련하면서 해당 열의 범주를 기반으로 원-핫 인코딩을 학습합니다.
- 검증 데이터셋(`x_valid`)에 대해 원-핫 인코딩을 수행합니다. 학습 데이터와 같은 열 순서 및 범주를 사용하여 변환합니다.

In [ ]:
from sklearn.preprocessing import ___

# OneHotEncoder 객체 생성
encoder = ___(sparse_output=False)

# 학습 데이터에 대해 fit_transform 실행
x_train_encoded = encoder.___(x_train[['race', 'sex', 'marital.status']])

# 검증 데이터에 대해 transform 실행
x_valid_encoded = encoder.transform(___[['race', 'sex', 'marital.status']])

In [65]:
#checkcode
import numpy as np
ensure_vals(globals(), 'x_train', 'encoder', 'x_train_encoded')
@check_safety
def check(
    encoder_name: str,
    enc: OneHotEncoder,
    col1: str,
    col2: str,
    onehoted: np.ndarray,
    indexing_col: int,
    len_col: int
):
    c_point0 = encoder_name in str(enc)
    c_point1 = col1 in enc.feature_names_in_
    c_point2 = col2 in enc.feature_names_in_
    c_point3 = onehoted.shape[indexing_col] == 13

    if (
        c_point0 and 
        c_point1 and
        c_point2 and 
        c_point3
    ):
        return True
    else:
        return False

check('OneHotEncoder', encoder, 'race', 'marital.status', x_train_encoded, 1, 13)

True

# 8.원-핫 인코딩된 데이터를 DataFrame으로 변환

scikit-learn의 OneHotEncoder가 반환하는 것은 numpy 배열 형태입니다.     
이는 기계 학습 모델에 바로 입력할 수 있는 형태이지만, 우리가 데이터를 직접 다루거나 분석할 때는 pandas DataFrame 형태가 더 편리합니다.     
DataFrame은 레이블링된 열(column)을 가지므로 어떤 열이 어떤 원래의 범주에 해당하는지 쉽게 파악할 수 있습니다.

따라서 원-핫 인코딩된 numpy 배열을 pandas DataFrame으로 변환해보도록 하겠습니다. 

[문제 7]  
`pd.DataFrame()` 메소드를 사용하여 원-핫 인코딩된 훈련 데이터를 새로운 데이터프레임으로 변환합니다.   
columns 인자에는 `get_feature_names_out()` 메소드를 사용하여 각각의 컬럼 이름을 생성합니다.

In [ ]:
x_train_ohe = pd.___(x_train_encoded, columns=encoder.___(['race', 'sex', 'marital.status']))
x_valid_ohe = pd.DataFrame(x_valid_encoded, columns=encoder.get_feature_names_out(['race', 'sex', 'marital.status']))

x_train_ohe.head(4)

In [72]:
#checkcode
ensure_vals(globals(), 'x_train_ohe')
@check_safety
def check(
    df: pd.DataFrame,
    shape_df: tuple
):
    c_point0 = df.shape == shape_df

    if c_point0:
        return True
    else:
        return False

check(x_train_ohe, (3108, 13))

True

# 9.데이터프레임에 원-핫 인코딩된 특성 추가

데이터 처리 과정에서는 종종 여러 데이터프레임을 합치는 작업이 필요합니다.   
이번에는 원-핫 인코딩된 데이터프레임(x_train_ohe, x_valid_ohe)과 원래의 데이터프레임(x_train, x_valid)을 합칠 것입니다.

pandas의 concat 함수를 사용하면 간단하게 두 데이터프레임을 합칠 수 있습니다.   
하지만 주의할 점은, concat 함수는 기본적으로 각 데이터프레임의 인덱스를 따라 동작한다는 것입니다.   
즉, 서로 다른 인덱스를 가진 두 데이터프레임을 합치게 되면 예상하지 못한 결과가 나올 수 있습니다.

따라서 우리는 먼저 reset_index 메소드를 사용하여 x_train과 x_valid의 인덱스를 초기화하겠습니다.    
인덱스가 초기화된 후에야 비로소 원-핫 인코딩된 피처들과 원래 피처들이 올바르게 연결시켜주겠습니다.



[문제 8]
- `reset_index(drop=True)`를 사용하여 기존의 인덱스를 삭제하고 새로운 연속적인 정수 인덱스로 설정해주세요.
- x_train_ohe와 x_valid_ohe를 `concat()` 함수를 사용하여   
각각의 기존 데이터프레임 x_train과 x_train_ohe을 열 방향(axis=1)으로 합쳐주세요.

In [319]:
x_train = x_train.___(drop=___)
x_valid = x_valid.reset_index(drop=True)

x_train = pd.___([x_train,x_train_ohe], axis=1)
x_valid = pd.concat([x_valid,x_valid_ohe], axis=1)

In [88]:
#checkcode
ensure_vals(globals(), 'x_train', 'x_valid')
@check_safety
def check(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    train_shape: tuple,
    val_shape: tuple
):
    c_point0 = train_df.shape == train_shape
    c_point1 = val_df.shape == val_shape

    if c_point0 and c_point1:
        return True
    else:
        return False

check(x_train, x_valid, (3108,26), (1332,26))

True

# 10.LabelEncoder를 사용한 범주형 데이터 인코딩

이전에 우리는 순서가 없는 범주형 피처들에 대해 원-핫 인코딩을 적용하였습니다.   
그러나 아직 처리하지 않은 다른 범주형 피처들도 있습니다.   
이들은 순서가 있는 데이터일 가능성이 있으므로, 레이블 인코딩(label encoding)을 적용하려 합니다.   
레이블 인코딩은 각 고유한 범주값에 대해 고유한 정수값을 할당하는 방식으로, 순서 정보를 유지할 수 있게 합니다.

[문제 9]   
- `LabelEncoder` 객체를 생성하여 le 변수에 할당해 주세요.
- `x_train` 데이터프레임의 현재 열에 LabelEncoder을 적용해 주세요.
- 만약 검증 데이터에서 새롭게 나타나는 범주값(label)이라면, 이를 인코더의 클래스 목록(le.classes_)에 추가해 주세요.
- `x_valid` 데이터프레임의 현재 열에 LabelEncoder을 적용해 주세요.

In [321]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

for col in x_train.columns:
    if x_train[col].dtype == 'object':
        
        le = ___()
        x_train[col] = ___.___(___[___])

        for label in np.unique(x_valid[col]):
            if label not in le.classes_:
                le.classes_ = np.___(le.classes_, ___)
        x_valid[col] = ___.___(___[___])

In [105]:
#checkcode
ensure_vals(globals(), 'x_train', 'x_valid')
@check_safety
def check(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    obj: str,
    zero: 0
):
    
    len_obj_train = len(train_df.select_dtypes(include=obj).columns)
    len_obj_test = len(val_df.select_dtypes(include=obj).columns)
    
    c_point0 = len_obj_train == zero
    c_point1 = len_obj_test == zero

    if c_point0:
        return True
    else:
        return False

check(x_train, x_valid, 'object', 0)

True

# 11.StandardScaler를 이용한 데이터 표준화

데이터 전처리는 머신러닝 모델의 성능에 중요한 역할을 합니다.   
이 과정에서 데이터의 특성들을 적절하게 변환하여 모델이 데이터를 더 잘 이해할 수 있도록 돕습니다.   
그 중 하나가 바로 특성 스케일링입니다.

특성들 간에 값의 범위가 다르면, 그 값이 큰 특성이 결과에 더 큰 영향을 주게 됩니다.   
예를 들어, '나이'는 보통 0-100 사이의 값을 가지지만, '수입'은 수 천에서 수 만까지 다양하게 분포할 수 있습니다.   
이런 경우 '수입' 특성이 결과에 과도하게 영향을 미치게 되므로, 모든 특성이 동등하게 반영되도록 스케일링 작업을 거치는 것입니다.

[문제 10]
- scikit-learn 라이브러리에서 제공하는 `StandardScaler` 클래스를 사용하여 scaler 객체를 생성해주세요.

- 학습 데이터셋(`x_train`)에 대해서는 `fit_transform` 메서드를 사용하여 평균과 분산을 계산(fit)한 후   
표준화(transform)를 진행합니다.

- 검증 데이터셋(`x_valid`)에 대해서는 `transform` 메서드를 사용하여   
학습 데이터셋에서 이미 계산된 평균과 분산 값을 사용하여 표준화(transform)를 진행합니다.   
이렇게 해야 학습과 검증 단계에서 일관된 전처리가 적용됩니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

features = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss','hours.per.week', 'age-hours']

scaler = ___()
x_train[features] = scaler.___(x_train[___])
x_valid[features] = ___.___(x_valid[features])

In [111]:
#checkcode
ensure_vals(globals(), 'x_train', 'x_valid', 'scaler')
@check_safety
def check(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    scaler_class: StandardScaler,
    num_features: int
):
    
    c_point0 = len(scaler_class.feature_names_in_) == num_features

    if c_point0:
        return True
    else:
        return False

check(x_train, x_valid, scaler, 7)

True

# 12.데이터 차원 축소를 위한 PCA 모델 학습

이번에는 주성분 분석(Principal Component Analysis, PCA)을 사용하여 데이터의 차원을 축소하는 작업을 수행해봅시다.   
PCA는 다차원 데이터를 저차원 공간으로 변환하여 데이터의 변동성을 최대한 보존하는 방식으로 동작합니다.

[문제 11]  
- `Scikit-learn`의 `decomposition` 모듈에서 `PCA` 클래스를 불러와 보세요.
- `PCA` 객체를 생성해보세요.
- 주어진 훈련 데이터인 `x_train에` 대해 `PCA` 모델을 적용(`fit`)시켜보세요. 

In [ ]:
from sklearn.___ import PCA

pca = PCA()
pca.___(x_train)

In [140]:
#checkcode
ensure_vals(globals(), 'x_train', 'pca')
@check_safety
def check(
    train_df: pd.DataFrame,
    pca_class
):
    
    c_point0 = len(train_df.columns) == len(pca_class.get_feature_names_out())
    
    if c_point0:
        return True
    else:
        return False

check(x_train, pca)

True

# 13.PCA 주성분 추출

In [141]:
pca.components_[0]

array([-4.79055732e-04,  2.88123473e-03,  7.14129075e-03,  2.93943613e-02,
       -1.19098206e-02,  9.96784763e-01, -5.38655166e-02,  5.70556703e-05,
        1.49095433e-02,  1.10344715e-02,  2.16331617e-03,  3.14708243e-02,
        2.42733518e-02,  6.85156857e-04,  7.17858710e-05, -1.44281378e-03,
       -1.27413157e-04,  8.13284206e-04, -1.49095433e-02,  1.49095433e-02,
       -1.29029930e-03, -1.05222737e-04,  7.49006552e-03, -9.83909787e-06,
       -3.66831033e-03, -2.41639405e-03])

# 14.PCA 주성분의 분산 설명량 확인

In [142]:
# 분산의 설명량 확인
explained_variance_ratio = pca.explained_variance_ratio_

print('explained_variance_ratio: \n',explained_variance_ratio)
print('\nexplained_variance_ratio[0]: ',explained_variance_ratio[0])
print('\nexplained_variance_ratio[1]: ',explained_variance_ratio[1])

explained_variance_ratio: 

 [4.94515588e-01 1.30375995e-01 7.19915105e-02 4.47434711e-02

 3.84268750e-02 3.59899981e-02 3.44973959e-02 3.34197805e-02

 3.08658290e-02 3.02436508e-02 2.74980741e-02 1.31476642e-02

 7.37395656e-03 2.32056637e-03 1.89438084e-03 1.12435680e-03

 5.87351407e-04 5.22480355e-04 4.16154596e-04 4.49208450e-05

 3.48816875e-32 1.90644454e-32 3.10229048e-33 3.10229048e-33

 3.10229048e-33 3.01745573e-33]



explained_variance_ratio[0]:  0.4945155881447502



explained_variance_ratio[1]:  0.13037599498893046


# 15.누적 설명량과 주성분 개수 선택

주성분 분석(PCA)은 데이터의 차원을 축소하기 위한 기술 중 하나입니다.   
보통, 축소할 차원을 임의로 정하기 보다는 충분한 분산이 될 때까지 더해야할 차원 수를 선택하는 쪽이 더 일반적입니다.   
이 방법을 사용하면 데이터의 정보를 최대한 보존하면서도 차원을 줄일 수 있습니다.   

`cumulative_explained_variance`를 계산하여 누적 설명량이 95% 이상이 되는 주성분 개수를 선택해보겠습니다.

[문제 12]  
- `cumsum` 함수를 사용하여 `explained_variance_ratio_` 값을 누적해서 더해주세요.
- `argmax` 함수를 사용하여 해당 임계값을 넘는 최초의 인덱스를 찾아보세요.

In [ ]:
# 누적 설명량 계산
cumulative_explained_variance = np.___(explained_variance_ratio)

# 적절한 주성분 개수 선택 (예: 95% 이상의 누적 설명량을 가지는 개수)
n_components = np.___(cumulative_explained_variance >= 0.95) + 1

In [152]:
#checkcode
import numpy as np
ensure_vals(globals(), 'x_train', 'cumulative_explained_variance')
@check_safety
def check(
    train_df: pd.DataFrame,
    variance: np.ndarray
):
    
    c_point0 = len(variance) == len(train_df.columns)
    
    if c_point0:
        return True
    else:
        return False

check(x_train, cumulative_explained_variance)

True

# 16.PCA 모델 재설정 및 데이터 변환

위에서 선택된 주성분 개수로 PCA 모델을 다시 초기화하고, 이전과 같은 데이터에 적용하여 차원을 축소합니다.   
이렇게 함으로써 데이터의 차원을 축소하면서 원본 데이터의 정보를 가능한 한 보존하게 됩니다.

[문제 14]  
- 주성분 분석(PCA) 모델을 초기화해주세요.   
이때, `n_components` 매개변수에는 이전 단계에서 선택한 주성분 개수인 `n_components`를 전달합니다.   

- 초기화된 PCA 모델을 기존의 훈련 데이터인 `x_train`에 다시 적용(`fit_transform`)해주세요.   
검증 데이터인 `x_valid`에 동일한 PCA 모델을 적용(`transform`)합니다.

In [ ]:
# 선택된 주성분 개수로 PCA 모델 다시 초기화
pca = ___(n_components=___)

# 데이터에 선택된 주성분 개수로 PCA 모델 적용
x_train = pca.___(___)
x_valid = pca.___(___)

In [166]:
#checkcode
import numpy as np
ensure_vals(globals(), 'pca')
@check_safety
def check(
    train_df: pd.DataFrame,
    pca_class: PCA
):
    
    c_point0 = len(pca_class.get_feature_names_out()) == pca_class.n_components
    
    if c_point0:
        return True
    else:
        return False

check(x_train, pca)

True

# 17.RandomForestClassifier 모델을 활용한 데이터 분석 및 성능 평가

`x_train`과 `x_valid` 데이터셋은 이미 PCA를 적용하여 주성분으로 변환된 데이터셋입니다. 

PCA 후에 남아 있는 각각의 주성분들은 원래 특성 공간에서 어떠한 방향(즉, 벡터)를 나타내며,   
이러한 방향들은 원래 데이터가 가장 많이 분산되어 있는 방향입니다.   
따라서 PCA로 변환된 `x_train`과 `x_valid`는 원래의 다차원 공간에서 가장 정보가 많이 담긴 방향으로 투영된 상태라고 볼 수 있습니다.

그런 다음 RandomForestClassifier를 사용하여 모델을 학습시킨 후  
마지막으로 F1 score를 계산하여 모델 성능을 평가하겠습니다.


[문제 15]

- RandomForestClassifier 모델을 생성해 주세요. random_state 는 42로 설정하여 재현성을 보장해주세요.
- 학습 데이터를 사용하여 학습시켜주세요.
- 학습된 모델을 사용하여 검증 데이터에 대한 예측을 수행합니다.
- f1_score 함수를 사용하여 실제 레이블인 검증 데이터의 목표값과 모델의 예측 결과를 비교하여 Macro F1 스코어를 계산해보세요. 

In [325]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf_classifier = ___(random_state=___)
___.fit(___, ___)

y_pred = ___.___(x_valid)

macro_f1 = f1_score(y_valid, y_pred, average='___')
print("Macro F1 스코어:", macro_f1)

Macro F1 스코어: 0.7929340560259506


In [181]:
#checkcode
import numpy as np
ensure_vals(globals(), 'x_train', 'y_train', 'rf_classifier', 'y_pred', 'macro_f1')
@check_safety
def check(
    x_train_df: pd.DataFrame,
    y_train_df: pd.DataFrame,
    x_valid_df: pd.DataFrame,
    val_pred: np.ndarray,
    model: RandomForestClassifier,
    model_name: str
):
    
    c_point0 = model_name in str(model)
    c_point1 = len(model.classes_) == y_train_df.nunique()
    c_point2 = len(val_pred) == len(x_valid_df)
    
    if c_point0:
        return True
    else:
        return False

check(x_train, y_train, x_valid, y_pred, rf_classifier, 'RandomForestClassifier')

True